# ibmpairs Python API Wrapper

## Overview

The ibmpairs Python API Wrapper (paw) is a python module that can be used to interact with the Geospatial Analytics component of the IBM Environmental Intelligence Suite (EIS). 

The wrapper is available to download from [PyPI](https://pypi.org/project/ibmpairs/).

### Layout and Versions

There are two versions of paw implemented across the various sub-modules that make up this library.

Version v0.1.x is in the sub-modules called `paw` and `utils` and only supports query operations. This version is retained to maintain backward compatibility for those still using it. 

<div class="alert alert-info">
The `paw` will be deprecated at some point in the future, 
so if you are new to Geospatial Analytics use version 0.2.x sub-modules instead. 
</div>

Version 0.2.x is embodied in all of the other sub-modules. The query function of Version 0.1.x was improved and separated into a sub-module called `query` and various other modules were added to give more complete coverage of the Geospatial Analytics API. 

| Sub-module | PAW Version | Description |
| :----- | :--------------- | :---------- |
| paw    | 0.1.x           | version 0.1.x of the paw library retained for backward compatibility |
| utils    | 0.1.x           | a utility package for paw |
|authentication | 0.2.x |sign in using basic auth or OAuth2 tokens |
| catalog | 0.2.x |allow an authorized Geospatial Analytics user to perform get, create, update, delete operations on Geospatial Analytics Data Sets, Data Layers, Data Layer Dimensions & Properties |
| client | 0.2.x |a common HTTP client for Geospatial Analytics which understands authentication from the module above |
| dashboard | 0.2.x |display the results of a Geospatial Analytics query on the Dashboard component |
| query | 0.2.x |creation, submission, monitoring and download of Geospatial Analytics queries and subsequent query data files. In addition, this module contains helper functions that allow for the retrieval of recent and favourite flagged queries and more |
| upload | 0.2.x |upload data to Geospatial Analytics |
| external/ibm | 0.2.x |interact with IBM COS (used by upload) |


## Model

The ibmpairs Python API Wrapper wraps the Geospatial Analytics server side HTTP API. The wrapper has been designed using a small number of patterns. 

At a very high level this is how it works:

```bash
Client Side Object <-------> Server Side Geospatial Analytics API
|
|-> Data Objects
```

i.e. Each client side object, such as Query or Catalog, wraps the server side Query and Catalog APIs. Data objects on the client side allow the data and parameters of the API calls to be provided without having to worry about the technical implementation of the API, for example, URL encoding. 

## Data Objects

Generally the client side objects are configured using data objects. Data objects can be expressed using JSON (or a Python dictionary), and inserted into the Geospatial Analytics objects with helper functions. e.g.:

```python
query_dict = {
  "layers": [
    {
      "id": "49464",
      "type": "raster"
    }
  ],
  "name": "Area Query Test",
  "spatial": {
    "coordinates": [
      48.437249,
      2.5735152,
      48.488131,
      2.6838934
    ],
    "type": "square"
  },
  "temporal": {
    "intervals": [
      {
        "end": "2016-11-08",
        "start": "2016-11-06"
      }
    ]
  }
}

q = query.query_from_dict(query_dict)

```

Alternatively a data object can be created directly and you can set its attributed individually in the init. 

```python
l = [{"id": "49464", "type": "raster"}]

q = query.Query(name   = "Area Query Test",
                layers = l)
```

The JSON responses from the Geospatial Analytics APIs will be converted to data objects and returned to you automatically.

If you want to change the data in a data object you can set the object's attributes:

```python
l = [{"id": "49464", "type": "raster"}, {"id": "12345", "type": "raster"}]
q.name = "New Name"
q.layers = l
```

## Reading from the API

The wrapper supports a number of different ways of reading data and meta-data from the Geospatial Analytics API. To read data or meta-data you need to provide its id.

### Approach 1 - Provide data id as an attribute to get()

```python
id = '1234'
data_layer = catalog.DataLayer()
data_layer.get(id = id)
```

### Approach 2 - Provide data id by constructing a client side object before calling get()

```python
id = '1234'
data_layer = catalog.DataLayer(id = id)
data_layer.get()
```

### Approach 3 - Helper Functions that hide get()

```python
id = '1234'
data_layer = catalog.get_data_layer(id)
```

## Writing to the API

The method used to "write" data to the Geospatial Analytics API varies depending on the data object in question. 
A data layer is created using a `create` operation and a query is run using a `submit` operation for example.
The wrapper supports a number of different patterns for writing client side objects to the server side:

### Approach 1 - Provide data by constructing a client side object before calling create()

```python
data_layer = catalog.DataLayer(crs         = "EPSG:4326",
                               datatype    = "bt",
                               level       = 21,
                               name        = "a_data_layer_name"
                              )
data_layer.create(data_set_id     = 1234, 
                  data_layer_type = "Raster"
                 )
```

The data_layer object will contact the Geospatial Analytics API and have it create the data layer assuming
you are authorized to do that. 

As an alternative to specifying separate fields the data_layer object can be created from JSON or a Python dictionary using the following pattern:

```python
data_layer = catalog.DataLayer.from_dict(<dictionary here>)
data_layer = catalog.DataLayer.from_json(<JSON string here>)
```

### Approach 2 - Helper Function hides create()

```python
layer_dictionary = {
  "data_layers": [
    {
      "color_table": {
        "id": 58
      },
      "crs": "EPSG:4326",
      "datatype": "bt",
      "level": 13,
      "name": "a_data_layer_name"
    }
  ],
  "layer_type": "Raster"
}
      
data_layers = catalog.data_layers_from_dict(layer_dictionary)
      
catalog.create_data_layers(data_layers            = data_layers
                           data_set_id            = 490, 
                           data_layer_type        = "Raster",
                          )
```

Using a DataLayers object rather than DataLayer is useful if you want to create many datalayers are the same time. 
This is usually the case when creating a dataset to represent a multi-band input data. 

## Using Client Objects

Once a client side object has been created it can be used for further processing. For example,

### Pass one Object to Another

An object within the library can sometimes be passed to another:

```python
credentials  = authentication.OAuth2()

#...

eis_client = client.Client(authentication = credentials)
```

Here we create a credentials object and use it as part of the construction of a client object.

### Pass an Object to the API

An object can sometimes be passed to a server side call:

```python
query = Query()
      
#...

dashboard.add_dashboard_layer(query)
```

In this example we want to display a query on the dashboard so we pass the query object to a helper method on the dashboard object which in turn forwards the query information on to the dashboard part of the Geospatial Analytics API.

### Use an Object Returned by the API

A returned result from a request is an object or list of objects and can be acted on:

```python
query_list = query.get_latest_queries()
query = query_list[0]
query.submit()
```

Here we get the latest queries that have been run and re-run the first one in the list. We might re-run a query if, for example, it uses as data layer that has been updated since we last ran the query.